Kaggle challenge - Spam vs Ham - https://www.kaggle.com/balakishan77/spam-or-ham-email-classification

In [1]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from sklearn.model_selection import train_test_split


     |████████████████████████████████| 4.9 MB 4.9 MB/s 
     |████████████████████████████████| 462 kB 45.3 MB/s 


Get Dataset

In [3]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding = "ISO-8859-1")
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df['Message'] = df['v2']
df['spam'] = df['v1'].apply(lambda x: 1 if x == "spam" else 0)
df = df.drop(columns = ['v1','v2','Unnamed: 2','Unnamed: 3','Unnamed: 4'])
df.head(5)

,Message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [11]:
### GET BERT Model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [12]:
## Train test split
X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [14]:
## build a model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
#l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(outputs['pooled_output'])

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=5)


Epoch 1/5
131/131 [==============================] - 88s 668ms/step - loss: 0.0815 - accuracy: 0.9777
Epoch 2/5
131/131 [==============================] - 87s 662ms/step - loss: 0.0804 - accuracy: 0.9785
Epoch 3/5
131/131 [==============================] - 86s 659ms/step - loss: 0.0767 - accuracy: 0.9799
Epoch 4/5
131/131 [==============================] - 86s 658ms/step - loss: 0.0740 - accuracy: 0.9809
Epoch 5/5
131/131 [==============================] - 86s 659ms/step - loss: 0.0738 - accuracy: 0.9797


In [18]:
model.evaluate(X_test, y_test)


44/44 [==============================] - 30s 660ms/step - loss: 0.0842 - accuracy: 0.9799


[0.08423115313053131, 0.979899525642395]

In [19]:
############## TEST out model


reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

array([[0.91175747],
       [0.94720674],
       [0.682256  ],
       [0.04902132],
       [0.00848683]], dtype=float32)

In [20]:
model.save("my_model")
#reconstructed_model = keras.models.load_model("my_model")

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [25]:
!zip -r /content/model1.zip /content/my_model

  adding: content/my_model/ (stored 0%)
  adding: content/my_model/.zip (stored 0%)
  adding: content/my_model/assets/ (stored 0%)
  adding: content/my_model/assets/vocab.txt (deflated 53%)
  adding: content/my_model/keras_metadata.pb (deflated 83%)
  adding: content/my_model/variables/ (stored 0%)
  adding: content/my_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/my_model/variables/variables.index (deflated 79%)
  adding: content/my_model/saved_model.pb (deflated 92%)


In [26]:
from google.colab import files
files.download("/content/model1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>